In [1]:
import random
import hoomd
import gsd.hoomd
from hoomd.wall import Sphere as WallSphere
from hoomd.md.manifold import Sphere as ManifoldSphere
from hoomd.md.methods.rattle import NVE
import os
import shutil
import numpy as np
import plotly.graph_objects as go

In [ ]:
pi = np.pi
seed = 42
notice_level = 10
np.random.seed(seed)
random.seed(seed)

In [ ]:
def random_point_on_sphere(radius):
    theta = np.random.uniform(0, 2 * np.pi)
    phi = np.arccos(1 - 2 * np.random.uniform())
    x = radius * np.sin(phi) * np.cos(theta)
    y = radius * np.sin(phi) * np.sin(theta)
    z = radius * np.cos(phi)
    return np.array([x, y, z])


def interpolate_points(start, end, num_points):
    vect = end - start
    unit_vect = vect / np.linalg.norm(vect)
    distances = np.linspace(0, np.linalg.norm(vect), num_points)
    return np.array([start + distance * unit_vect for distance in distances])


def place_polymer_in_sphere(radius, num_particles):
    telo1_pos = random_point_on_sphere(radius)
    telo2_pos = random_point_on_sphere(radius)
    while np.linalg.norm(telo1_pos - telo2_pos) < radius:
        telo2_pos = random_point_on_sphere(radius)

    polymer_positions = interpolate_points(telo1_pos, telo2_pos, num_particles - 2)
    polymer_positions = np.vstack([telo1_pos, polymer_positions, telo2_pos])
    return polymer_positions


def plot_polymer(polymer_positions, sizes, radius):

    fig = go.Figure()
    u, v = np.mgrid[0:2 * np.pi:100j, 0:np.pi:50j]
    x = radius * np.cos(u) * np.sin(v)
    y = radius * np.sin(u) * np.sin(v)
    z = radius * np.cos(v)

    fig.add_trace(go.Mesh3d(
        x=x.flatten(),
        y=y.flatten(),
        z=z.flatten(),
        color='lightpink',
        opacity=0.1,
        alphahull=0
    ))

    start = 0
    colors = ['red', 'green', 'blue']
    c = -1
    for s, size in enumerate(sizes):
        positions = polymer_positions[start:start+size]
        start += size
        if s % 2 == 0:
            c += 1
        fig.add_trace(go.Scatter3d(
            x=positions[:, 0],
            y=positions[:, 1],
            z=positions[:, 2],
            mode='lines+markers',
            marker=dict(size=4, color=colors[c]),
            line=dict(color=colors[c], width=2)
        ))

    fig.update_layout(
        scene=dict(
            xaxis=dict(nticks=4, range=[-radius - 1, radius + 1]),
            yaxis=dict(nticks=4, range=[-radius - 1, radius + 1]),
            zaxis=dict(nticks=4, range=[-radius - 1, radius + 1]),
            aspectmode='cube'
        ),
        width=1920,
        height=1080,
        margin=dict(r=20, l=10, b=10, t=10)
    )

    fig.show()

In [ ]:
cwd = os.getcwd()
data = os.path.join(os.path.dirname(cwd), 'data')
snapshots_dir = os.path.join(os.path.dirname(cwd), 'snapshots')
lattice_init_path = os.path.join(snapshots_dir, "lattice_init.gsd")

simu_id = "test"
simu_dir = os.path.join(data, simu_id)
if os.path.exists(simu_dir):
    shutil.rmtree(simu_dir)
os.makedirs(simu_dir)
os.makedirs(snapshots_dir, exist_ok=True)
if os.path.exists(lattice_init_path):
    os.remove(lattice_init_path)

In [ ]:
n_poly = 6
poly_sizes = [24, 24, 36, 36, 48, 48]
poly_sizes_single = [24, 36, 48]
nb_breaks = 8  # nb of DNA breaks
timepoints = 500    # total number of timestep
dt = 0.0001    # timestep
radius = 12.0
L = 3*radius
calibration_time = 10
min_distance_binding = 10.2  # required dist for homologous binding
persistence_length = 10
period = 1000   # periodic trigger
mode = 'cpu'    # run on cpu or gpu
rd_pos = False  # place the polymer randomly or not
debug = False   # debug mode (useful on IDE)

In [ ]:
particles_positions = []
for size in poly_sizes:
    positions = place_polymer_in_sphere(radius - 0.5, size)
    particles_positions.extend(positions)
particles_positions = np.array(particles_positions)

In [ ]:
plot_polymer(particles_positions, poly_sizes, radius)

In [ ]:
n_particles = len(particles_positions)
particles_ids = list(range(n_particles))
particles_types = ['dna', 'tel', 'dsb']
particles_typeid = np.array([1 if i == 0 or i == s - 1 else 0 for s in poly_sizes for i in range(s)])

homologous_pairs = []
counter = 0
for s, size in enumerate(poly_sizes_single):
    for ll in range(size):
        homologous_pairs.append([counter + ll, counter + ll + size])
    counter += 2 * size
homologous_pairs = np.array(homologous_pairs)

n_bonds = n_particles - n_poly
bonds_group = np.zeros((n_bonds, 2), dtype=int)
bonds_types = ["dna-telo", "dna-dna", "dna-dsb", "dsb-dsb"]
bonds_typeid = np.array(
    [bonds_types.index("dna-telo") if i == 0 or i == size - 2 else bonds_types.index("dna-dna")
     for size in poly_sizes for i in range(size - 1)])

n_angles = n_particles - n_poly * 2
angles_group = np.zeros((n_angles, 3), dtype=int)
angles_types = ["dna-dna-dna", "dna-dsb-dna"]
angles_typeid = np.array([angles_types.index("dna-dna-dna") for size in poly_sizes for i in range(size - 2)])

# Optimizing loops for bonds and angles
start_indices = list(np.cumsum([0] + poly_sizes[:-1]))
b_counter = 0
a_counter = 0
for x, start in enumerate(start_indices):
    x_len = poly_sizes[x]
    for b in range(x_len - 1):
        bonds_group[b_counter] = [start + b, start + b + 1]
        b_counter += 1
    for a in range(x_len - 2):
        angles_group[a_counter] = [start + a, start + a + 1, start + a + 2]
        a_counter += 1

breaks_ids = np.random.choice(particles_ids, nb_breaks, replace=False)
particles_typeid[breaks_ids] = particles_types.index('dsb')
homologous_breaks_pairs = homologous_pairs[np.where(np.isin(homologous_pairs, breaks_ids))[0]]


In [ ]:
frame = gsd.hoomd.Frame()
frame.particles.N = n_particles
frame.particles.types = particles_types
frame.particles.typeid = particles_typeid
frame.particles.position = particles_positions

frame.bonds.N = n_bonds
frame.bonds.group = bonds_group
frame.bonds.typeid = bonds_typeid
frame.bonds.types = bonds_types

frame.angles.N = n_angles
frame.angles.group = angles_group
frame.angles.typeid = angles_typeid
frame.angles.types = angles_types

frame.configuration.box = [L, L, L, 0, 0, 0]
frame.configuration.dimensions = 3

In [2]:
# The system state (aka a "snapshot") defined above is then used to initialize a HOOMD-blue simulation.
try:
    current_device = hoomd.device.GPU()
    print(current_device.get_available_devices(), current_device.is_available())
except:
    current_device = hoomd.device.CPU()
    print('No GPU found, using CPU')

['[0]NVIDIA T1200 Laptop GPU  16 SM_7.5 @ 1.43 GHz, 3906 MiB DRAM'] True
